# Denoising Autoencoder

In [1]:
import NBInclude: @nbinclude
@nbinclude("../Helpers/Base.ipynb");

In [2]:
# train_alpha(
#     create_hyperparams(
#         "autoencoder",
#         "explicit",
#         ["ExplicitUserItemBiases"],
#         ["ExplicitUserItemBiases"],
#     ),
#     "NeuralExplicitAutoencoderUntuned";
#     tune_hyperparams=false
# )

In [3]:
# train_alpha(
#     create_hyperparams(
#         "autoencoder",
#         "implicit",
#         ["NeuralImplicitUserItemBiases"],
#         ["ExplicitUserItemBiases"],
#     ),
#     "NeuralImplicitAutoencoder";
#     tune_hyperparams=false    
# )

In [4]:
# train_alpha(
#     create_hyperparams(
#         "autoencoder",
#         "ptw",
#         ["NeuralPtwUserItemBiases"],
#         ["ExplicitUserItemBiases"],
#     ),
#     "NeuralPtwAutoencoder";
#     tune_hyperparams=false
# )

In [5]:
spl = "validation"
truth = get_split(spl, "explicit")
df = read_alpha(
    ["ExplicitUserItemBiases", "ExplicitItemCF"], # ExplicitItemCF NeuralExplicitAutoencoder
    spl,
    "explicit",
    false,
) #
df = @set df.rating = (truth.rating .- df.rating) .^ 2

RatingsDataset
  user: Array{Int32}((33587462,)) Int32[17, 17, 17, 17, 17, 17, 17, 17, 17, 17  …  4209008, 4209024, 4209040, 4209072, 4209120, 4209152, 4209216, 4209232, 4209248, 4209264]
  item: Array{Int32}((33587462,)) Int32[16600, 11645, 11146, 12651, 8815, 2583, 15511, 16407, 14091, 4807  …  16335, 18470, 3710, 1307, 10390, 20865, 2603, 7293, 14558, 4625]
  rating: Array{Float32}((33587462,)) Float32[0.97151923, 2.9742644, 0.03295037, 0.14887871, 1.2430565, 0.0148444045, 4.006312, 0.31815046, 2.806966, 0.0016622362  …  1.3223631, 11.896229, 2.4351578, 2.9577777, 0.39902687, 0.50482416, 1.5585995, 0.0009878003, 0.022250528, 0.11354707]
  timestamp: Array{Float32}((0,)) Float32[]
  user_timestamp: Array{Float32}((0,)) Float32[]
  item_timestamp: Array{Float32}((0,)) Float32[]
  status: Array{Int32}((0,)) Int32[]
  completion: Array{Float32}((0,)) Float32[]
  rewatch: Array{Int32}((0,)) Int32[]
  source: Array{Int32}((0,)) Int32[]
  order: Array{Int32}((0,)) Int32[]


In [6]:
using Statistics

In [7]:
user_frac = df.user ./ maximum(df.user);

In [8]:
function per_user_mse(df)
    errors = zeros(Float32, num_users())
    counts = zeros(Float32, num_users())

    for i = 1:length(df.rating)
        errors[df.user[i]] += df.rating[i]
        counts[df.user[i]] += 1
    end

    Threads.@threads for i = 1:length(errors)
        if counts[i] > 0
            errors[i] /= counts[i]
        end
    end
    errors, counts
end

per_user_mse (generic function with 1 method)

In [9]:
tcounts = get_counts("training", "explicit"; per_rating = false);

In [10]:
split_df = df
mses, counts = per_user_mse(split_df)
mean(mses[(counts.>0)]),
mean(mses[@. (counts > 0) & (tcounts > 10)]),
mean(mses[@. (counts > 0) & (tcounts > 50)])

(1.6689993f0, 1.4876989f0, 1.4150063f0)

In [11]:
split_df = filter(df, @. (user_frac <= 0.95))
mses, counts = per_user_mse(split_df)
mean(mses[(counts.>0)]),
mean(mses[@. (counts > 0) & (tcounts > 10)]),
mean(mses[@. (counts > 0) & (tcounts > 50)])

(1.6481068f0, 1.4799522f0, 1.4070175f0)

In [12]:
split_df = filter(df, @. (user_frac >= 0.95))
mses, counts = per_user_mse(split_df)
mean(mses[(counts.>0)]),
mean(mses[@. (counts > 0) & (tcounts > 10)]),
mean(mses[@. (counts > 0) & (tcounts > 50)])

(2.0778043f0, 1.6553136f0, 1.5737627f0)

In [13]:
for s = 0.90:0.01:0.99
    split_df = filter(df, @. (s <= user_frac) & (user_frac < s + 0.01))
    mses, counts = per_user_mse(split_df)
    @info s,
    mean(mses[(counts.>0)]),
    mean(mses[@. (counts > 0) & (tcounts > 10)]),
    mean(mses[@. (counts > 0) & (tcounts > 50)])
end


[ Info: 20220923 19:04:29 (0.9, 1.6156317f0, 1.4649644f0, 1.3977793f0)
[ Info: 20220923 19:04:30 (0.91, 1.6157233f0, 1.4754889f0, 1.4168503f0)
[ Info: 20220923 19:04:30 (0.92, 1.6374185f0, 1.4716161f0, 1.4074227f0)
[ Info: 20220923 19:04:30 (0.93, 1.6365209f0, 1.4522483f0, 1.3853385f0)
[ Info: 20220923 19:04:30 (0.94, 1.6434923f0, 1.4762218f0, 1.419723f0)
[ Info: 20220923 19:04:30 (0.95, 2.00032f0, 1.4622947f0, 1.3875049f0)
[ Info: 20220923 19:04:30 (0.96, 1.9072773f0, 1.4860296f0, 1.4519022f0)
[ Info: 20220923 19:04:30 (0.97, 2.0101247f0, 1.7765248f0, 1.6535605f0)
[ Info: 20220923 19:04:30 (0.98, 2.2671297f0, 1.7706509f0, 1.6809839f0)
[ Info: 20220923 19:04:30 (0.99, 2.2207909f0, 1.8087268f0, 1.7398107f0)


In [14]:
filter(df, @. (user_frac >= 0.95))

RatingsDataset
  user: Array{Int32}((604349,)) Int32[3998817, 3998833, 3998833, 3998833, 3998833, 3998849, 3998849, 3998849, 3998849, 3998849  …  4209008, 4209024, 4209040, 4209072, 4209120, 4209152, 4209216, 4209232, 4209248, 4209264]
  item: Array{Int32}((604349,)) Int32[2422, 19877, 5050, 18470, 9651, 1193, 6260, 7025, 10653, 18543  …  16335, 18470, 3710, 1307, 10390, 20865, 2603, 7293, 14558, 4625]
  rating: Array{Float32}((604349,)) Float32[0.2999705, 0.82871425, 0.82713497, 5.554381, 0.033942394, 2.2327185, 0.47066125, 17.382626, 0.35383016, 5.582892  …  1.3223631, 11.896229, 2.4351578, 2.9577777, 0.39902687, 0.50482416, 1.5585995, 0.0009878003, 0.022250528, 0.11354707]
  timestamp: Array{Float32}((0,)) Float32[]
  user_timestamp: Array{Float32}((0,)) Float32[]
  item_timestamp: Array{Float32}((0,)) Float32[]
  status: Array{Int32}((0,)) Int32[]
  completion: Array{Float32}((0,)) Float32[]
  rewatch: Array{Int32}((0,)) Int32[]
  source: Array{Int32}((0,)) Int32[]
  order: Array{I